In [1]:
from docplex.mp.model import Model
from docplex.mp.utils import DOcplexException
from qiskit import Aer
from qiskit.algorithms import NumPyEigensolver, VQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import RealAmplitudes
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit_optimization import translators

from qiskit_polynomial_program.polynomial_program import PolynomialProgram

In [2]:
def compute_eigenvalues(qubit_op):
    count = 2 ** qubit_op.num_qubits
    eigensolver = NumPyEigensolver(count)
    eigensolver_result = eigensolver.compute_eigenvalues(qubit_op)
    print('state\teigenvalue')
    for eigenstate, eigenvalue in zip(eigensolver_result.eigenstates, eigensolver_result.eigenvalues):
        eigenstate, = eigenstate.sample().keys()
        eigenvalue = eigenvalue
        print(f'{eigenstate}\t{eigenvalue}')

# Quadratic functions

## `QuadraticProgram`

In [3]:
algorithm_globals.random_seed = 12345678

model = Model(name='sample')
x = {i: model.binary_var(name=f'x_{i}') for i in range(3)}
objective = x[0] + x[1] ** 2 - x[2]
model.minimize(objective)

quadratic_program = translators.from_docplex_mp(model)
hamiltonian_quadratic, offset = quadratic_program.to_ising()

compute_eigenvalues(hamiltonian_quadratic)
print('offset:', offset)

state	eigenvalue
100	(-1.5+0j)
000	(-0.5+0j)
101	(-0.5+0j)
110	(-0.5+0j)
001	(0.5+0j)
010	(0.5+0j)
111	(0.5+0j)
011	(1.5+0j)
offset: 0.5


### VQE

In [4]:
optimizer = COBYLA()
ry = RealAmplitudes(hamiltonian_quadratic.num_qubits, reps=1, entanglement='full')
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, seed_simulator=algorithm_globals.random_seed,
                                   seed_transpiler=algorithm_globals.random_seed)
vqe = VQE(ry, optimizer=optimizer, quantum_instance=quantum_instance)
result = vqe.compute_minimum_eigenvalue(hamiltonian_quadratic)

for state, amplitude in result.eigenstate.items():
    state = state[::-1]
    print(state, amplitude ** 2)

000 0.004882812500000001
010 0.0019531250000000004
001 0.9931640625000001


### QAOA

In [5]:
# TODO

## `PolynomialProgram`

In [6]:
algorithm_globals.random_seed = 12345678

polynomial_program = PolynomialProgram(3)
x = polynomial_program.x
polynomial_program.add_objective(x[0] + x[1] ** 2 - x[2])
hamiltonian_polynomial, offset = polynomial_program.to_ising()
compute_eigenvalues(hamiltonian_polynomial)
print('offset:', offset)

state	eigenvalue
100	(-1.5+0j)
000	(-0.5+0j)
101	(-0.5+0j)
110	(-0.5+0j)
001	(0.5+0j)
010	(0.5+0j)
111	(0.5+0j)
011	(1.5+0j)
offset: 0.5


### VQE

In [7]:
optimizer = COBYLA()
ry = RealAmplitudes(hamiltonian_polynomial.num_qubits, reps=1, entanglement='full')
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, seed_simulator=algorithm_globals.random_seed,
                                   seed_transpiler=algorithm_globals.random_seed)
vqe = VQE(ry, optimizer=optimizer, quantum_instance=quantum_instance)
result = vqe.compute_minimum_eigenvalue(hamiltonian_polynomial)

for state, amplitude in result.eigenstate.items():
    state = state[::-1]
    print(state, amplitude ** 2)

000 0.004882812500000001
010 0.0019531250000000004
001 0.9931640625000001


### QAOA

In [8]:
#TODO

# Higher order functions


## `QuadraticProgram`

In [9]:
algorithm_globals.random_seed = 1234567

model = Model(name='sample')
x = {i: model.binary_var(name=f'x_{i}') for i in range(3)}
try:
    objective = x[0] * x[1] * x[2]
    model.minimize(objective)

    quadratic_program = translators.from_docplex_mp(model)
    hamiltonian_quadratic, offset = quadratic_program.to_ising()

    compute_eigenvalues(hamiltonian_quadratic)
    print('offset:', offset)
except DOcplexException as e:
    print('Failed to create Hamiltonian')
    print(e)

Failed to create Hamiltonian
Cannot multiply x_0*x_1 by x_2, some terms would have degree >= 3. Maximum polynomial degree is 2.


# `PolynomialProgram`

In [10]:
algorithm_globals.random_seed = 12345678

polynomial_program = PolynomialProgram(3)
x = polynomial_program.x
polynomial_program.add_objective(x[0] * x[1] * x[2])
hamiltonian_polynomial, offset = polynomial_program.to_ising()
compute_eigenvalues(hamiltonian_polynomial)
print('offset:', offset)

state	eigenvalue
000	(-0.125+0j)
001	(-0.125+0j)
010	(-0.125+0j)
011	(-0.125+0j)
100	(-0.125+0j)
101	(-0.125+0j)
110	(-0.125+0j)
111	(0.875+0j)
offset: 0.125


### VQE

In [11]:
optimizer = COBYLA()
ry = RealAmplitudes(hamiltonian_polynomial.num_qubits, reps=1, entanglement='full')
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, seed_simulator=algorithm_globals.random_seed,
                                   seed_transpiler=algorithm_globals.random_seed)
vqe = VQE(ry, optimizer=optimizer, quantum_instance=quantum_instance)
result = vqe.compute_minimum_eigenvalue(hamiltonian_polynomial)

for state, amplitude in result.eigenstate.items():
    state = state[::-1]
    print(state, amplitude ** 2)

000 0.5498046875000001
100 0.19921875000000003
010 0.1435546875
110 0.012695312499999998
001 0.0537109375
101 0.0166015625
011 0.0244140625


### QAOA

In [12]:
#TODO